# August Perez Capstone Three Project:
## Characterization of High Accident Situations

Using neural networks / deep learning I plan on analyzing a conglomerate dataset to characterize what combined variables of road types, times, and conditions have the highest probability of a car accident occuring.

### Goal
Build models for characterization of locations with crash probability of >=80% (or top 10 if minimal quantity of >=80% crash probability)
and an analysis of times, days of the week, and dates of the year with crash probability >=60% for the locations characterized.

    Crash probability Thresholds chosen arbitrarily. For real-world application, each agency utilizing this needs to determine thresholds specific to their needs and capabilities.

### Data source:
Accidents in France from 2005 to 2016 (https://www.kaggle.com/datasets/ahmedlahlou/accidents-in-france-from-2005-to-2016/data)


    
#### About the dataset:
- A collection of 5 datasets pertaining to car crashes in France from 2005 to 2016
    - characteristics
        - Details about each crash
    - holidays
        - Dates from 2005 to 2016 that are holidays
    - places
        - Details about accident locations
    - users
        - Details about persons involved in the accident
    - vehicles
        - Details about vehicles involved in the accident

## Imports:

In [1]:
%matplotlib inline

# data manipulation and math

import numpy as np
import scipy as sp
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

# plotting and visualization

import matplotlib.pyplot as plt
import seaborn as sns

# modeling & pre-processing
    #commenting out since not necessary for this part of the project
#    import sklearn.model_selection
#    from sklearn.model_selection import train_test_split
#    from sklearn.model_selection import KFold
#    import sklearn.preprocessing
#    import sklearn.metrics

In [2]:
plt.rcParams['figure.figsize'] = [8,8]

### Set random seed for reproducability
Note that this should not be done for models used in real-world applications

In [3]:
np.random.seed(9)

## Load the data into a pandas df's

### Column Descriptions
Large section. Suggested to keep collapsed.

CARACTERISTICS :

Num_Acc : Accident ID

jour : Day of the accident

mois : Month of the accident

an : Year of the accident

hrmn : Time of the accident in hour and minutes (hhmm)

lum : Lighting : lighting conditions in which the accident occurred

    1 - Full day

    2 - Twilight or dawn

    3 - Night without public lighting

    4 - Night with public lighting not lit

    5 - Night with public lighting on

dep : Departmeent : INSEE Code (National Institute of Statistics and Economic Studies) of the departmeent followed
by a 0 (201 Corse-du-Sud - 202 Haute-Corse)

com : Municipality: The commune number is a code given by INSEE. The code has 3 numbers set to the right.

Localisation :

    1 - Out of agglomeration

    2 - In built-up areas

int : Type of Intersection :

    1 - Out of intersection

    2 - Intersection in X

    3 - Intersection in T

    4 - Intersection in Y

    5 - Intersection with more than 4 branches

    6 - Giratory

    7 - Place

    8 - Level crossing

    9 - Other intersection

atm : Atmospheric conditions:

    1 - Normal

    2 - Light rain

    3 - Heavy rain

    4 - Snow - hail

    5 - Fog - smoke

    6 - Strong wind - storm

    7 - Dazzling weather

    8 - Cloudy weather

    9 - Other

col : Type of collision:

    1 - Two vehicles - frontal

    2 - Two vehicles - from the rear

    3 - Two vehicles - by the side

    4 - Three vehicles and more - in chain

    5 - Three or more vehicles - multiple collisions

    6 - Other collision

    7 - Without collision

adr : Postal address: variable filled in for accidents occurring in built-up areas

gps : GPS coding: 1 originator character:

    M = Métropole

    A = Antilles (Martinique or Guadeloupe)

    G = Guyane

    R = Réunion

    Y = Mayotte

Geographic coordinates in decimal degrees:

    lat : Latitude

    long : Longitude

Places:

Num_Acc : Accident ID

catr : Category of road:

    1 - Highway

    2 - National Road

    3 - Departmental Road

    4 - Communal Way

    5 - Off public network

    6 - Parking lot open to public traffic

    9 - other

voie : Road Number

V1: Numeric index of the route number (example: 2 bis, 3 ter etc.)

V2: Letter alphanumeric index of the road

circ: Traffic regime:

    1 - One way

    2 - Bidirectional

    3 - Separated carriageways

    4 - With variable assignment channels

nbv: Total number of traffic lanes

vosp: Indicates the existence of a reserved lane, regardless of whether or not the accident occurs on that lane.

    1 - Bike path

    2 - Cycle Bank

    3 - Reserved channel

Prof: Longitudinal profile describes the gradient of the road at the accident site

    1 - Dish

    2 - Slope

    3 - Hilltop

    4- Hill bottom

pr: Home PR number (upstream terminal number)

pr1: Distance in meters to the PR (relative to the upstream terminal)

plan: Drawing in plan:

    1 - Straight part

    2 - Curved on the left

    3 - Curved right

    4 - In "S"

lartpc: Central solid land width (TPC) if there is

larrout: Width of the roadway assigned to vehicle traffic are not included the emergency stop strips,
CPRs and parking spaces

surf: surface condition

    1 - normal

    2 - wet

    3 - puddles

    4 - flooded

    5 - snow

    6 - mud

    7 - icy

    8 - fat - oil

    9 - other

infra: Development - Infrastructure:

    1 - Underground - tunnel

    2 - Bridge - autopont

    3 - Exchanger or connection brace

    4 - Railway

    5 - Carrefour arranged

    6 - Pedestrian area

    7 - Toll zone

situ: Situation of the accident:

    1 - On the road

    2 - On emergency stop band

    3 - On the verge

    4 - On the sidewalk

    5 - On bike path

env1: school point: near a school

USERS:

Acc_number: Accident identifier.

Num_Veh: Identification of the vehicle taken back for each user occupying this vehicle (including pedestrians who are
attached to the vehicles that hit them)

place: Allows to locate the place occupied in the vehicle by the user at the time of the accident

catu: User category:

    1 - Driver

    2 - Passenger

    3 - Pedestrian

    4 - Pedestrian in rollerblade or scooter

grav: Severity of the accident: The injured users are classified into three categories of victims plus the uninjured

    1 - Unscathed

    2 - Killed

    3 - Hospitalized wounded

    4 - Light injury

sex: Sex of the user

    1 - Male

    2 - Female

Year_on: Year of birth of the user

trip: Reason for traveling at the time of the accident:

    1 - Home - work

    2 - Home - school

    3 - Shopping - Shopping

    4 - Professional use

    5 - Promenade - leisure

    9 - Other

secu: on 2 characters:
the first concerns the existence of a safety equipment

    1 - Belt

    2 - Helmet

    3 - Children's device

    4 - Reflective equipment

    9 - Other

the second is the use of Safety Equipment

    1 - Yes

    2 - No

    3 - Not determinable

locp: Location of the pedestrian:

On pavement:

    1 - A + 50 m from the pedestrian crossing

    2 - A - 50 m from the pedestrian crossing

On pedestrian crossing:

    3 - Without light signaling

    4 - With light signaling

Various:

    5 - On the sidewalk

    6 - On the verge

    7 - On refuge or BAU

    8 - On against aisle

actp: Action of the pedestrian:

Moving

    0 - not specified or not applicable

    1 - Meaning bumping vehicle

    2 - Opposite direction of the vehicle
    Various

    3 - Crossing

    4 - Masked

    5 - Playing - running

    6 - With animal

    9 - Other

etatp: This variable is used to specify whether the injured pedestrian was alone or not

    1 - Only

    2 - Accompanied

    3 - In a group

VEHICLES:

Num_Acc
Accident ID

Num_Veh
Identification of the vehicle taken back for each user occupying this vehicle (including pedestrians who are
attached to vehicles that hit them) - alphanumeric code

GP
Flow direction :

    1 - PK or PR or increasing postal address number

    2 - PK or PR or descending postal address number

CATV
Category of vehicle:

    01 - Bicycle

    02 - Moped <50cm3

    03 - Cart (Quadricycle with bodied motor) (formerly "cart or motor tricycle")

    04 - Not used since 2006 (registered scooter)

    05 - Not used since 2006 (motorcycle)

    06 - Not used since 2006 (side-car)

    07 - VL only

    08 - Not used category (VL + caravan)

    09 - Not used category (VL + trailer)

    10 - VU only 1,5T <= GVW <= 3,5T with or without trailer (formerly VU only 1,5T <= GVW <= 3,5T)

    11 - Most used since 2006 (VU (10) + caravan)

    12 - Most used since 2006 (VU (10) + trailer)

    13 - PL only 3,5T


### Characteristics Data

In [4]:
#AP: To get encoding of the csv since it's not the default (because French source of data)
with open(r'Datasets_cap3\characteristics.csv') as f:
    print(f)

<_io.TextIOWrapper name='Datasets_cap3\\characteristics.csv' mode='r' encoding='cp1252'>


In [5]:
df_char = pd.read_csv(r'Datasets_cap3\characteristics.csv', skip_blank_lines=True, encoding='cp1252', encoding_errors='ignore', low_memory=False)

In [6]:
#AP: Should have 839,985 rows, based on looking at csv
print(len(df_char))

839985


### holidays Data

In [7]:
#AP: To get encoding of the csv since it's not the default (because French source of data)
with open(r'Datasets_cap3\holidays.csv') as f:
    print(f)

<_io.TextIOWrapper name='Datasets_cap3\\holidays.csv' mode='r' encoding='cp1252'>


In [8]:
df_holiday = pd.read_csv(r'Datasets_cap3\holidays.csv', skip_blank_lines=True, encoding='cp1252', encoding_errors='ignore', low_memory=False)

In [9]:
#AP: Should have 132 rows, based on looking at csv
print(len(df_holiday))

132


### places Data

In [10]:
#AP: To get encoding of the csv since it's not the default (because French source of data)
with open(r'Datasets_cap3\places.csv') as f:
    print(f)

<_io.TextIOWrapper name='Datasets_cap3\\places.csv' mode='r' encoding='cp1252'>


In [11]:
df_places = pd.read_csv(r'Datasets_cap3\places.csv', skip_blank_lines=True, encoding='cp1252', encoding_errors='ignore', low_memory=False)

In [12]:
#AP: Should have 839,985 rows, based on looking at csv
print(len(df_places))

839985


### users Data

Had to rename file to 'crash_users.csv' from 'users.csv'because of unicode escape character error

In [13]:
#AP: To get encoding of the csv since it's not the default (because French source of data)
with open(r'Datasets_cap3\crash_users.csv') as f:
    print(f)

<_io.TextIOWrapper name='Datasets_cap3\\crash_users.csv' mode='r' encoding='cp1252'>


In [14]:
df_users = pd.read_csv(r'Datasets_cap3\crash_users.csv', skip_blank_lines=True, encoding='cp1252', encoding_errors='ignore', low_memory=False)

In [15]:
#AP: Should have about 1.88 million rows, based on kaggle stats
print(len(df_users))

1876005


### vehicles Data

Had to rename file to 'crash_vehicles.csv' from 'ehicles.csv'because of unicode character error

AP: Not using this dataset. Research into each of the features suggests to me that they would not be beneficial in characterizing crash locations for purposes of predicting where they might occur.

If investigation into this data is needed, convert the cells back to code cells (currently as Raw so the code doesn't execute)

# Data Wrangling

DF var names:
- df_char
    - (characteristics)
- df_holiday
    - (holidays)
- df_places
    - (places)
- df_users
    - (users)
- df_vehicles
    - (vehicles)

## Initial Exploration

In [16]:
df_char.head()

,Num_Acc,an,mois,jour,hrmn,lum,agg,int,atm,col,com,adr,gps,lat,long,dep
0,201600000001,16,2,1,1445,1,2,1,8.0,3.0,5.0,"46, rue Sonneville",M,0.0,0,590
1,201600000002,16,3,16,1800,1,2,6,1.0,6.0,5.0,1a rue du cimetière,M,0.0,0,590
2,201600000003,16,7,13,1900,1,1,1,1.0,6.0,11.0,NaN,M,0.0,0,590
3,201600000004,16,8,15,1930,2,2,1,7.0,3.0,477.0,52 rue victor hugo,M,0.0,0,590
4,201600000005,16,12,23,1100,1,2,3,1.0,3.0,11.0,rue Joliot curie,M,0.0,0,590


In [17]:
df_char.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 839985 entries, 0 to 839984
Data columns (total 16 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   Num_Acc  839985 non-null  int64  
 1   an       839985 non-null  int64  
 2   mois     839985 non-null  int64  
 3   jour     839985 non-null  int64  
 4   hrmn     839985 non-null  int64  
 5   lum      839985 non-null  int64  
 6   agg      839985 non-null  int64  
 7   int      839985 non-null  int64  
 8   atm      839930 non-null  float64
 9   col      839974 non-null  float64
 10  com      839983 non-null  float64
 11  adr      699443 non-null  object 
 12  gps      366226 non-null  object 
 13  lat      362471 non-null  float64
 14  long     362467 non-null  object 
 15  dep      839985 non-null  int64  
dtypes: float64(4), int64(9), object(3)
memory usage: 102.5+ MB


In [18]:
df_holiday.head()

,ds,holiday
0,2005-01-01,New year
1,2005-03-28,Easter Monday
2,2005-05-01,Labour Day
3,2005-05-05,Ascension Thursday
4,2005-05-08,Victory in Europe Day


In [19]:
df_holiday.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 132 entries, 0 to 131
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   ds       132 non-null    object
 1   holiday  132 non-null    object
dtypes: object(2)
memory usage: 2.2+ KB


In [20]:
df_places.head()

,Num_Acc,catr,voie,v1,v2,circ,nbv,pr,pr1,vosp,prof,plan,lartpc,larrout,surf,infra,situ,env1
0,201600000001,3.0,39,NaN,NaN,2.0,0.0,NaN,NaN,0.0,1.0,3.0,0.0,0.0,1.0,0.0,1.0,0.0
1,201600000002,3.0,39,NaN,NaN,1.0,0.0,NaN,NaN,0.0,1.0,2.0,0.0,58.0,1.0,0.0,1.0,0.0
2,201600000003,3.0,1,NaN,NaN,2.0,2.0,NaN,NaN,0.0,1.0,3.0,0.0,68.0,2.0,0.0,3.0,99.0
3,201600000004,4.0,0,NaN,NaN,2.0,0.0,NaN,NaN,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,99.0
4,201600000005,4.0,0,NaN,NaN,0.0,0.0,NaN,NaN,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,3.0


In [21]:
df_places.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 839985 entries, 0 to 839984
Data columns (total 18 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   Num_Acc  839985 non-null  int64  
 1   catr     839984 non-null  float64
 2   voie     780914 non-null  object 
 3   v1       332816 non-null  float64
 4   v2       33953 non-null   object 
 5   circ     839187 non-null  float64
 6   nbv      838195 non-null  float64
 7   pr       414770 non-null  float64
 8   pr1      413463 non-null  float64
 9   vosp     838345 non-null  float64
 10  prof     838924 non-null  float64
 11  plan     838909 non-null  float64
 12  lartpc   830440 non-null  float64
 13  larrout  831706 non-null  float64
 14  surf     838968 non-null  float64
 15  infra    838707 non-null  float64
 16  situ     838983 non-null  float64
 17  env1     838709 non-null  float64
dtypes: float64(15), int64(1), object(2)
memory usage: 115.4+ MB


In [22]:
df_users.head()

,Num_Acc,place,catu,grav,sexe,trajet,secu,locp,actp,etatp,an_nais,num_veh
0,201600000001,1.0,1,1,2,0.0,11.0,0.0,0.0,0.0,1983.0,B02
1,201600000001,1.0,1,3,1,9.0,21.0,0.0,0.0,0.0,2001.0,A01
2,201600000002,1.0,1,3,1,5.0,11.0,0.0,0.0,0.0,1960.0,A01
3,201600000002,2.0,2,3,1,0.0,11.0,0.0,0.0,0.0,2000.0,A01
4,201600000002,3.0,2,3,2,0.0,11.0,0.0,0.0,0.0,1962.0,A01


In [23]:
df_users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1876005 entries, 0 to 1876004
Data columns (total 12 columns):
 #   Column   Dtype  
---  ------   -----  
 0   Num_Acc  int64  
 1   place    float64
 2   catu     int64  
 3   grav     int64  
 4   sexe     int64  
 5   trajet   float64
 6   secu     float64
 7   locp     float64
 8   actp     float64
 9   etatp    float64
 10  an_nais  float64
 11  num_veh  object 
dtypes: float64(7), int64(4), object(1)
memory usage: 171.8+ MB


## Edit Col names
    For clarity & some translation from French

    Making new col names with orginal appended at end (for my own ease when referring to dataset documentation)

### df_char

In [24]:
df_char.head(2)

,Num_Acc,an,mois,jour,hrmn,lum,agg,int,atm,col,com,adr,gps,lat,long,dep
0,201600000001,16,2,1,1445,1,2,1,8.0,3.0,5.0,"46, rue Sonneville",M,0.0,0,590
1,201600000002,16,3,16,1800,1,2,6,1.0,6.0,5.0,1a rue du cimetière,M,0.0,0,590


In [25]:
df_char.rename({'Num_Acc' : 'acc_id',
                'an' : 'year_an',
                'mois' : 'month_mois',
                'jour' : 'day_jour',
                'hrmn' : 'hrmn',
                'lum' : 'luminosity_lum',
                'agg' : 'built_up_agg',
                'int' : 'intersection_type_int',
                'atm' : 'weather_atm',
                'col' : 'collision_type_col',
                'adr' : 'address_drop',
                'gps' : 'gps_drop',
                'lat' : 'latitude_drop',
                'long' : 'longitude_drop',
                'dep' : 'dep_drop',
                'com' : 'commune_num_drop'},
               axis=1, inplace=True)
df_char.head(2)

,acc_id,year_an,month_mois,day_jour,hrmn,luminosity_lum,built_up_agg,intersection_type_int,weather_atm,collision_type_col,commune_num_drop,address_drop,gps_drop,latitude_drop,longitude_drop,dep_drop
0,201600000001,16,2,1,1445,1,2,1,8.0,3.0,5.0,"46, rue Sonneville",M,0.0,0,590
1,201600000002,16,3,16,1800,1,2,6,1.0,6.0,5.0,1a rue du cimetière,M,0.0,0,590


### df_holiday

In [26]:
df_holiday.head(2)

,ds,holiday
0,2005-01-01,New year
1,2005-03-28,Easter Monday


In [27]:
list(df_holiday.holiday.unique())

['New year',
 'Easter Monday',
 'Labour Day',
 'Ascension Thursday',
 'Victory in Europe Day',
 'Whit Monday',
 'Bastille Day',
 'Assumption of Mary to Heaven',
 'All Saints Day',
 'Armistice Day',
 'Christmas Day']

#AP: Note that these are French holidays, keeping since this dataset is about French crashes

In [28]:
df_holiday.rename({'ds' : 'date'}, axis=1, inplace=True)
print(list(df_holiday.columns))

['date', 'holiday']


### df_places

In [29]:
df_places.sample(2)

,Num_Acc,catr,voie,v1,v2,circ,nbv,pr,pr1,vosp,prof,plan,lartpc,larrout,surf,infra,situ,env1
448123,200900013184,3.0,900,NaN,NaN,2.0,2.0,34.0,293.0,0.0,1.0,2.0,0.0,54.0,1.0,0.0,3.0,99.0
451153,200900016214,4.0,0,NaN,NaN,3.0,2.0,5.0,0.0,0.0,1.0,1.0,15.0,70.0,1.0,0.0,1.0,99.0


In [30]:
df_places.env1.value_counts()

env1
0.0     477933
99.0    319986
3.0      40790
Name: count, dtype: int64

In [31]:
df_places.rename({'Num_Acc' : 'acc_id',
                  'catr' : 'road_category_catr',
                  'voie' : 'road_num_drop',
                  'v1' : 'v1_drop',
                  'v2' : 'v2_drop',
                  'circ' : 'road_type_circ',
                  'nbv' : 'lane_count_nbv',
                  'vosp' : 'reserved_lane_type_vosp',
                  'prof' : 'road_slope_prof',
                  'pr' : 'pr_drop',
                  'pr1' : 'pr1_drop',
                  'plan' : 'road_curvature_plan',
                  'lartpc' : 'central_sep_width_lartpc',
                  'larrout' : 'road_width_larrout',
                  'surf' : 'surface_cond_surf',
                  'infra' : 'infrastructure_infra',
                  'situ' : 'crash_location_situ',
                  'env1' : 'env1_drop'}, axis=1, inplace=True)
print(list(df_places.columns))

['acc_id', 'road_category_catr', 'road_num_drop', 'v1_drop', 'v2_drop', 'road_type_circ', 'lane_count_nbv', 'pr_drop', 'pr1_drop', 'reserved_lane_type_vosp', 'road_slope_prof', 'road_curvature_plan', 'central_sep_width_lartpc', 'road_width_larrout', 'surface_cond_surf', 'infrastructure_infra', 'crash_location_situ', 'env1_drop']


In [32]:
df_places.head(2)

,acc_id,road_category_catr,road_num_drop,v1_drop,v2_drop,road_type_circ,lane_count_nbv,pr_drop,pr1_drop,reserved_lane_type_vosp,road_slope_prof,road_curvature_plan,central_sep_width_lartpc,road_width_larrout,surface_cond_surf,infrastructure_infra,crash_location_situ,env1_drop
0,201600000001,3.0,39,NaN,NaN,2.0,0.0,NaN,NaN,0.0,1.0,3.0,0.0,0.0,1.0,0.0,1.0,0.0
1,201600000002,3.0,39,NaN,NaN,1.0,0.0,NaN,NaN,0.0,1.0,2.0,0.0,58.0,1.0,0.0,1.0,0.0


dropping env1 because values don't make sense and no var of that name in documentation from French Gov.t

### df_users

In [33]:
df_users.head(2)

,Num_Acc,place,catu,grav,sexe,trajet,secu,locp,actp,etatp,an_nais,num_veh
0,201600000001,1.0,1,1,2,0.0,11.0,0.0,0.0,0.0,1983.0,B02
1,201600000001,1.0,1,3,1,9.0,21.0,0.0,0.0,0.0,2001.0,A01


In [34]:
df_users.rename({'Num_Acc' : 'acc_id',
                 'num_veh' : 'num_veh_drop',
                 'place' : 'place_in_veh_place',
                 'catu' : 'person_cat_catu',
                 'grav' : 'injury_severity_grav',
                 'sexe' : 'sexe_drop',
                 'trajet' : 'travel_reason_trajet',
                 'secu' : 'secu_drop',
                 'locp' : 'ped_location_locp',
                 'actp' : 'ped_action_actp',
                 'etatp' : 'ped_group_etatp',
                 'an_nais' : 'an_nais_drop'}, axis=1, inplace=True)
print(list(df_users.columns))

['acc_id', 'place_in_veh_place', 'person_cat_catu', 'injury_severity_grav', 'sexe_drop', 'travel_reason_trajet', 'secu_drop', 'ped_location_locp', 'ped_action_actp', 'ped_group_etatp', 'an_nais_drop', 'num_veh_drop']


In [35]:
df_users.head(2)

,acc_id,place_in_veh_place,person_cat_catu,injury_severity_grav,sexe_drop,travel_reason_trajet,secu_drop,ped_location_locp,ped_action_actp,ped_group_etatp,an_nais_drop,num_veh_drop
0,201600000001,1.0,1,1,2,0.0,11.0,0.0,0.0,0.0,1983.0,B02
1,201600000001,1.0,1,3,1,9.0,21.0,0.0,0.0,0.0,2001.0,A01


### df_vehicles

## Drop unneeded cols

Dropping cols that I am reasonably sure would not benefit characterization of accident locations or date/time analysis for the use of preemptively deploying resources/personnel

ex. year of birth may have an effect on probability of a crash but agencies are unable to pre-screen drivers before they get on the road or determine the exact route the driver will take

In [36]:
df_char.drop(['commune_num_drop', 'address_drop', 'gps_drop', 'latitude_drop', 'longitude_drop', 'dep_drop'], axis=1, inplace=True, errors= 'ignore')
print(list(df_char.columns))

['acc_id', 'year_an', 'month_mois', 'day_jour', 'hrmn', 'luminosity_lum', 'built_up_agg', 'intersection_type_int', 'weather_atm', 'collision_type_col']


In [37]:
df_places.drop(['road_num_drop', 'v1_drop', 'v2_drop', 'pr_drop', 'pr1_drop', 'env1_drop'], axis=1, inplace=True, errors= 'ignore')
print(list(df_places.columns))

['acc_id', 'road_category_catr', 'road_type_circ', 'lane_count_nbv', 'reserved_lane_type_vosp', 'road_slope_prof', 'road_curvature_plan', 'central_sep_width_lartpc', 'road_width_larrout', 'surface_cond_surf', 'infrastructure_infra', 'crash_location_situ']


In [38]:
df_users.drop(['sexe_drop', 'secu_drop', 'an_nais_drop', 'num_veh_drop'], axis=1, inplace=True, errors= 'ignore')
print(list(df_users.columns))

['acc_id', 'place_in_veh_place', 'person_cat_catu', 'injury_severity_grav', 'travel_reason_trajet', 'ped_location_locp', 'ped_action_actp', 'ped_group_etatp']


#AP: dropping 'sexe' col b/c not looking to differentiate between sexes for this project

## Merge df's where possible

In [39]:
print('Row count of each df:')
for df in [df_char, df_holiday, df_places, df_users,
           # df_vehicles
          ]:
    print(len(df))

Row count of each df:
839985
132
839985
1876005


#AP: df_char & df_places have same row count, look into if they actually match

#AP: Explore 'acc_id' col to make sure it's actually unique per accident

In [40]:
match_acc_id = df_char[df_char.acc_id == df_places.acc_id].acc_id
no_match_acc_id = df_char[df_char.acc_id != df_places.acc_id].acc_id

In [41]:
print(len(match_acc_id))
print(len(no_match_acc_id))

839985
0


#AP: accident ID's seem to match. Joining the tables.

In [42]:
df_char_place = df_char.merge(df_places, on='acc_id', how='inner')

In [43]:
print('df_char:\n', list(df_char.columns))
print()
print('df_places:\n', list(df_places.columns))
print()
print(list(df_char_place.columns))
print()
print('row count:', len(df_char_place))

df_char:
 ['acc_id', 'year_an', 'month_mois', 'day_jour', 'hrmn', 'luminosity_lum', 'built_up_agg', 'intersection_type_int', 'weather_atm', 'collision_type_col']

df_places:
 ['acc_id', 'road_category_catr', 'road_type_circ', 'lane_count_nbv', 'reserved_lane_type_vosp', 'road_slope_prof', 'road_curvature_plan', 'central_sep_width_lartpc', 'road_width_larrout', 'surface_cond_surf', 'infrastructure_infra', 'crash_location_situ']

['acc_id', 'year_an', 'month_mois', 'day_jour', 'hrmn', 'luminosity_lum', 'built_up_agg', 'intersection_type_int', 'weather_atm', 'collision_type_col', 'road_category_catr', 'road_type_circ', 'lane_count_nbv', 'reserved_lane_type_vosp', 'road_slope_prof', 'road_curvature_plan', 'central_sep_width_lartpc', 'road_width_larrout', 'surface_cond_surf', 'infrastructure_infra', 'crash_location_situ']

row count: 839985


### Explore df_vehicles to try & merge
It has a acc_id col but more rows than df_char_place. Want to see if I can reduce row count to match. This project doesn't need specific vehicle info

The count of unique acc_id's seem to match. Now to see if plausible to aggregate so row counts match

Dropping vehicle_cat_catv col since multiple labels unused since 2006 present (in this 2005-2016 dataset)

AP: **Researching deeper into each of the features in the vehicles df, they do not seem that they would be helpful in characterizing where accidents occur for the purposes of predicting where they would occur.**

Converting df_vehicles cells to raw so the code doesn't run and errors are avoided

### Explore df_users to try & merge

There are multiple people per crash, trying to see if there is a way to group by acc_id in a way that doesn't loose quality of data.

In [44]:
df_users.head(10)

,acc_id,place_in_veh_place,person_cat_catu,injury_severity_grav,travel_reason_trajet,ped_location_locp,ped_action_actp,ped_group_etatp
0,201600000001,1.0,1,1,0.0,0.0,0.0,0.0
1,201600000001,1.0,1,3,9.0,0.0,0.0,0.0
2,201600000002,1.0,1,3,5.0,0.0,0.0,0.0
3,201600000002,2.0,2,3,0.0,0.0,0.0,0.0
4,201600000002,3.0,2,3,0.0,0.0,0.0,0.0
5,201600000003,1.0,1,1,1.0,0.0,0.0,0.0
6,201600000003,NaN,3,3,5.0,6.0,2.0,1.0
7,201600000004,1.0,1,3,5.0,0.0,0.0,0.0
8,201600000004,2.0,2,4,0.0,0.0,0.0,0.0
9,201600000004,1.0,1,1,0.0,0.0,0.0,0.0


In [45]:
len(df_users)

1876005

In [46]:
df_users.acc_id.nunique(dropna=False)

839985

____
**Drop place_in_veh_place col**. Not useful in characterizing locations

____
person_cat_catu col

In [47]:
df_users.person_cat_catu.value_counts(dropna=False).sort_index()

person_cat_catu
1    1397701
2     319388
3     155649
4       3267
Name: count, dtype: int64

AP: Note: There should only be values 1, 2, or 3 for this col. There are 3267 rows with value of 4.

**AP Note: Possible to create col to denote whether pedestrian present or not (0 : No or 1 : Yes). Presence of persons in vehicle is assumed since dataset is about vehicle crashes.**

Also possible to create col for count of persons involved. This could help agencies decide how much resources/personnel to deploy to the area in relation to how many people they expect to treat. This is currently outside the scope of the project.

____
injury_severity_grav col

In [48]:
df_users.injury_severity_grav.value_counts(dropna=False).sort_index()

injury_severity_grav
1    764874
2     50589
3    393669
4    666873
Name: count, dtype: int64

AP: **Possible to create col that ranks severity of the accident based on total injuries. Would likely subtract 1 from each value in col (since 1 represents unharmed, converting it to zero would keep the final score more easily understood). A higher score would mean more severe accident in terms of people harmed, requiring more resources from the agency to deal with.**

**This could also be used as a tie breaker between locations with equal accident probabilities.**

____
travel_reason_trajet col

In [49]:
df_users.travel_reason_trajet.value_counts(dropna=False).sort_index()

travel_reason_trajet
0.0    550516
1.0    242558
2.0     37816
3.0     47419
4.0    184032
5.0    686902
9.0    126393
NaN       369
Name: count, dtype: int64

This col is fairly vague (over 550 thousand rows have unspecified route).

**Dropping since not helpful for crash location characterization**

____
ped_location_locp

In [50]:
df_users.ped_location_locp.value_counts(dropna=False).sort_index()

ped_location_locp
0.0    1732439
1.0      22841
2.0      36711
3.0      27838
4.0      40011
5.0       9737
6.0       3060
7.0        173
8.0       1531
NaN       1664
Name: count, dtype: int64

In [51]:
print('Percent of total rows where ped location unspecified or not applicable')
print(round(df_users.ped_location_locp.value_counts(dropna=False).sort_index()[0] / len(df_users), 2) * 100, '%')

Percent of total rows where ped location unspecified or not applicable
92.0 %


With about 92% of df_user rows with value of 0 (unspecified ped location), this col is not helpful for location characterization

**dropping col**

____
ped_action_actp

In [52]:
df_users.ped_action_actp.value_counts(dropna=False).sort_index()

ped_action_actp
0.0    1723439
1.0       8441
2.0       4215
3.0     115011
4.0       2809
5.0       9282
6.0        347
9.0      10688
NaN       1773
Name: count, dtype: int64

In [53]:
len(df_users)

1876005

In [54]:
print(round(df_users.ped_action_actp.value_counts(dropna=False).sort_index()[0] / len(df_users), 2) * 100, '%')

92.0 %


With about 92% of df_user rows with value of 0 (unspecified ped location), this col is not helpful for location characterization

**dropping col**

____
ped_group_etatp

In [55]:
df_users.ped_group_etatp.value_counts(dropna=False).sort_index()

ped_group_etatp
0.0    1725093
1.0     113063
2.0      29597
3.0       6546
NaN       1706
Name: count, dtype: int64

In [56]:
print(round(df_users.ped_group_etatp.value_counts(dropna=False).sort_index()[0] / len(df_users), 2) * 100, '%')

92.0 %


With about 92% of df_user rows with value of 0 (unspecified ped location), this col is not helpful for location characterization

**dropping col**

#### Drop noted cols

In [57]:
df_users.drop(axis=1, inplace=True, errors= 'ignore', labels=['place_in_veh_place', 'travel_reason_trajet', 'ped_location_locp', 'ped_action_actp', 'ped_group_etatp'])
list(df_users.columns)

['acc_id', 'person_cat_catu', 'injury_severity_grav']

#### Create cols for aggregation

In [58]:
df_users.head()

,acc_id,person_cat_catu,injury_severity_grav
0,201600000001,1,1
1,201600000001,1,3
2,201600000002,1,3
3,201600000002,2,3
4,201600000002,2,3


In [59]:
df_users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1876005 entries, 0 to 1876004
Data columns (total 3 columns):
 #   Column                Dtype
---  ------                -----
 0   acc_id                int64
 1   person_cat_catu       int64
 2   injury_severity_grav  int64
dtypes: int64(3)
memory usage: 42.9 MB


In [60]:
#AP: Double check for any nulls
df_users.isnull().sum()

acc_id                  0
person_cat_catu         0
injury_severity_grav    0
dtype: int64

____
from person_cat_catu, create col to denote 0 (No) or 1 (Yes) for pedestrian

In [61]:
df_users.person_cat_catu.value_counts().sort_index()

person_cat_catu
1    1397701
2     319388
3     155649
4       3267
Name: count, dtype: int64

Value of 3 represents pedestrian. Want to replace 3 with 1 & all other values with 0

In [62]:
df_users['ped_present'] = df_users.person_cat_catu.replace({1:0, 2:0, 3:1, 4:0})

In [63]:
#AP: Check whether I Created col correctly
print(df_users[df_users['person_cat_catu']==3].sample(5))
print()
print(df_users[df_users['person_cat_catu'].isin([1,2,4])].sample(5))

               acc_id  person_cat_catu  injury_severity_grav  ped_present
1262581  200800058588                3                     4            1
1500505  200600004168                3                     4            1
1573682  200600036746                3                     3            1
704308   201100017939                3                     4            1
226992   201500041979                3                     3            1

               acc_id  person_cat_catu  injury_severity_grav  ped_present
1507954  200600007303                1                     1            0
1867263  200500083352                1                     1            0
1507846  200600007274                2                     4            0
583953   201200025973                2                     4            0
747405   201100037651                2                     4            0


**Now when I group by the acc_id col, I can just sum up how the pedestrians and treat that as applicable in further analysis.**

____
from injury_severity_grav, create col to denote accident severity based on injuries

First to create col with each value reduced by 1.

Durring grouping/aggregation, can add the adjusted values together to get an accident severity score.

In [64]:
df_users['injury_severity_adj_grav'] = df_users['injury_severity_grav'] -1

In [65]:
df_users[['injury_severity_grav', 'injury_severity_adj_grav']].sample(5)

,injury_severity_grav,injury_severity_adj_grav
669353,4,3
77361,1,0
1127285,1,0
794700,4,3
976514,3,2


### Create grouped/aggregated df for merging into the main df

In [66]:
df_users_agg = df_users[['acc_id', 'ped_present', 'injury_severity_adj_grav']].groupby(by='acc_id', sort=False, dropna=False, as_index=False).sum()

In [67]:
df_users_agg.head(15)

,acc_id,ped_present,injury_severity_adj_grav
0,201600000001,0,2
1,201600000002,0,6
2,201600000003,1,2
3,201600000004,0,5
4,201600000005,0,4
5,201600000006,1,2
6,201600000007,0,3
7,201600000008,0,2
8,201600000009,0,2
9,201600000010,1,2


In [68]:
df_users.head(15)

,acc_id,person_cat_catu,injury_severity_grav,ped_present,injury_severity_adj_grav
0,201600000001,1,1,0,0
1,201600000001,1,3,0,2
2,201600000002,1,3,0,2
3,201600000002,2,3,0,2
4,201600000002,2,3,0,2
5,201600000003,1,1,0,0
6,201600000003,3,3,1,2
7,201600000004,1,3,0,2
8,201600000004,2,4,0,3
9,201600000004,1,1,0,0


In [69]:
df_users_agg.ped_present.value_counts(dropna=False).sort_index()

ped_present
0     693464
1     139020
2       6415
3        773
4        214
5         55
6         19
7         13
8          2
9          3
10         4
15         1
21         1
25         1
Name: count, dtype: int64

In [70]:
df_users_agg.injury_severity_adj_grav.value_counts().sort_index()

injury_severity_adj_grav
1       28794
2      239710
3      393143
4       29629
5       34345
        ...  
114         1
129         1
136         1
141         1
178         1
Name: count, Length: 74, dtype: int64

In [71]:
len(df_users_agg)

839985

Change col name of ped_present to ped_count for clarity

Change col name of injury_severity_adj_grav to injury_count for clarity

In [72]:
df_users_agg.rename(columns={'ped_present':'ped_count', 'injury_severity_adj_grav':'injury_count'}, inplace=True)
df_users_agg.head()

,acc_id,ped_count,injury_count
0,201600000001,0,2
1,201600000002,0,6
2,201600000003,1,2
3,201600000004,0,5
4,201600000005,0,4


AP: Everything seems in place for merging df_users_agg into the main df

### Merge into main df

In [73]:
df_main = df_char_place.merge(df_users_agg, on='acc_id', how='inner')

In [74]:
print('df_char_place row count:', len(df_char_place))
print()
print('df_users_agg row count:', len(df_users_agg))
print()
print('df_main row count:', len(df_main))

df_char_place row count: 839985

df_users_agg row count: 839985

df_main row count: 839985


In [75]:
df_main.head(10)

,acc_id,year_an,month_mois,day_jour,hrmn,luminosity_lum,built_up_agg,intersection_type_int,weather_atm,collision_type_col,road_category_catr,road_type_circ,lane_count_nbv,reserved_lane_type_vosp,road_slope_prof,road_curvature_plan,central_sep_width_lartpc,road_width_larrout,surface_cond_surf,infrastructure_infra,crash_location_situ,ped_count,injury_count
0,201600000001,16,2,1,1445,1,2,1,8.0,3.0,3.0,2.0,0.0,0.0,1.0,3.0,0.0,0.0,1.0,0.0,1.0,0,2
1,201600000002,16,3,16,1800,1,2,6,1.0,6.0,3.0,1.0,0.0,0.0,1.0,2.0,0.0,58.0,1.0,0.0,1.0,0,6
2,201600000003,16,7,13,1900,1,1,1,1.0,6.0,3.0,2.0,2.0,0.0,1.0,3.0,0.0,68.0,2.0,0.0,3.0,1,2
3,201600000004,16,8,15,1930,2,2,1,7.0,3.0,4.0,2.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0,5
4,201600000005,16,12,23,1100,1,2,3,1.0,3.0,4.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0,4
5,201600000006,16,12,23,1115,1,2,1,7.0,6.0,3.0,2.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,1,2
6,201600000007,16,5,1,1145,1,2,1,7.0,2.0,3.0,2.0,0.0,0.0,1.0,1.0,70.0,0.0,1.0,0.0,1.0,0,3
7,201600000008,16,5,14,1915,2,1,1,1.0,1.0,3.0,2.0,2.0,0.0,1.0,1.0,0.0,70.0,1.0,0.0,1.0,0,2
8,201600000009,16,9,23,1900,1,2,1,1.0,3.0,4.0,2.0,2.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0,2
9,201600000010,16,12,30,1030,1,1,1,9.0,6.0,4.0,2.0,0.0,0.0,0.0,1.0,0.0,44.0,7.0,0.0,1.0,1,2


## Create cols for holiday & yes/no if holiday

Both to analyze if holidays have more accidents and also analyze which holidays have increased accidents.

In [76]:
df_holiday.head(12) #because 11 holidays

,date,holiday
0,2005-01-01,New year
1,2005-03-28,Easter Monday
2,2005-05-01,Labour Day
3,2005-05-05,Ascension Thursday
4,2005-05-08,Victory in Europe Day
5,2005-05-16,Whit Monday
6,2005-07-14,Bastille Day
7,2005-08-15,Assumption of Mary to Heaven
8,2005-11-01,All Saints Day
9,2005-11-11,Armistice Day


In [77]:
df_holiday.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 132 entries, 0 to 131
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   date     132 non-null    object
 1   holiday  132 non-null    object
dtypes: object(2)
memory usage: 2.2+ KB


In [78]:
df_holiday['date'] = pd.to_datetime(df_holiday['date'])

In [79]:
df_holiday.holiday.value_counts()

holiday
New year                        12
Easter Monday                   12
Labour Day                      12
Ascension Thursday              12
Victory in Europe Day           12
Whit Monday                     12
Bastille Day                    12
Assumption of Mary to Heaven    12
All Saints Day                  12
Armistice Day                   12
Christmas Day                   12
Name: count, dtype: int64

____
Create a 'date' col in datetime dtype

In [80]:
df_main['date'] = pd.to_datetime(df_main[['year_an', 'month_mois', 'day_jour']]
                                 .rename(columns={'year_an': 'year', 'month_mois': 'month', 'day_jour': 'day'})
                                 .assign(year=lambda x: x['year']+2000))

In [81]:
df_main.head()

,acc_id,year_an,month_mois,day_jour,hrmn,luminosity_lum,built_up_agg,intersection_type_int,weather_atm,collision_type_col,road_category_catr,road_type_circ,lane_count_nbv,reserved_lane_type_vosp,road_slope_prof,road_curvature_plan,central_sep_width_lartpc,road_width_larrout,surface_cond_surf,infrastructure_infra,crash_location_situ,ped_count,injury_count,date
0,201600000001,16,2,1,1445,1,2,1,8.0,3.0,3.0,2.0,0.0,0.0,1.0,3.0,0.0,0.0,1.0,0.0,1.0,0,2,2016-02-01
1,201600000002,16,3,16,1800,1,2,6,1.0,6.0,3.0,1.0,0.0,0.0,1.0,2.0,0.0,58.0,1.0,0.0,1.0,0,6,2016-03-16
2,201600000003,16,7,13,1900,1,1,1,1.0,6.0,3.0,2.0,2.0,0.0,1.0,3.0,0.0,68.0,2.0,0.0,3.0,1,2,2016-07-13
3,201600000004,16,8,15,1930,2,2,1,7.0,3.0,4.0,2.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0,5,2016-08-15
4,201600000005,16,12,23,1100,1,2,3,1.0,3.0,4.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0,4,2016-12-23


____
Create a col 'is_holiday' of 0/1 (No, Yes) for whether or not it's a holiday.

In [82]:
df_main['is_holiday'] = df_main['date'].isin(df_holiday['date'])

In [83]:
df_main[['date', 'is_holiday']].sample(10)

,date,is_holiday
184247,2013-02-15,False
772752,2005-04-27,False
220513,2013-11-24,False
109193,2015-10-05,False
117328,2015-02-06,False
268257,2012-12-11,False
329271,2011-07-06,False
616195,2007-05-31,False
29886,2016-10-17,False
145478,2014-08-23,False


____
Create a col 'holiday' for noting which holiday is is.

For non-holidays, input value 'common_day'

In [ ]:
df_holiday = df_holiday.sort_values(by='date', ascending=False)
df_holiday.head(11)

In [ ]:
df_main.merge(df_holiday, how='outer', on='date').sort_values(by='acc_id')

# EDA